In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
PATH_TO_TRAIN = 'processeddata/dataOrderMoreThanSame3TrainT2.csv'
PATH_TO_TEST = 'processeddata/dataOrderMoreThanSame3TestT2.csv'
from keras.models import Sequential
from keras.layers import Dense,BatchNormalization,Dropout,Reshape,Flatten,Embedding,Merge
from keras.callbacks import ModelCheckpoint
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from keras.models import load_model  
from keras.models import model_from_json
from keras.layers.convolutional import Conv2D,MaxPooling2D,Conv1D,MaxPooling1D


Using TensorFlow backend.


## Training data

In [4]:

"""1.读取数据"""
data = pd.read_csv(PATH_TO_TRAIN)
#data=data.loc[data.loc[:,"Time"]!=0.0]#去掉是0的
valid = pd.read_csv(PATH_TO_TEST)
#valid=valid.loc[valid.loc[:,"Time"]!=0.0]#去掉是0的
### START CODE HERE ### (≈ 3 lines of code)
"""2.训练集用userIDx和ItemIdx替换userID 和ItemID"""
m_train = len(data) #92979
m_test = len(valid) #6887
#train data 前处理
items=data["ItemId"].unique()# 不加unique为 92979
itemsnum=len(items)
print("itemsnum",itemsnum)#2341
itemidmap = pd.Series(data=np.arange(itemsnum), index=items)#把itemID转换成int map
rightmap= pd.DataFrame({"ItemId":items, 'ItemIdx':itemidmap[items].values})
#print("rightmap",rightmap)
data = pd.merge(data,rightmap, on="ItemId", how='left')#left??
users=data["UserId"].unique()# 不加unique为 92979
usersnum=len(users)
print("usersnum",usersnum)#1976
useridmap = pd.Series(data=np.arange(usersnum), index=users)#把itemID转换成int map
userrightmap= pd.DataFrame({"UserId":users, 'UserIdx':useridmap[users].values})
#print("rightmap",rightmap)
data = pd.merge(data,userrightmap, on="UserId", how='left')#left??
del(data["ItemId"])
del(data["UserId"])
del(data["Unnamed: 0"])
del(data["Unnamed: 0.1"])
#Quantity        Time    SessionId  ItemIdx  UserIdx
#print(data) 
#data.to_csv("itemidxuseridxdata.csv")
### END CODE HERE ###
"""3.测试集用userIDx和ItemIdx替换userID 和ItemID"""
#test data 前处理
items=valid["ItemId"].unique()# 不加unique为 92979
itemsnum=len(items)
print("itemsnum",itemsnum)#2341
itemidmap = pd.Series(data=np.arange(itemsnum), index=items)#把itemID转换成int map
rightmap= pd.DataFrame({"ItemId":items, 'ItemIdx':itemidmap[items].values})
#print("rightmap",rightmap)
valid = pd.merge(valid,rightmap, on="ItemId", how='left')#left??
users=valid["UserId"].unique()# 不加unique为 92979
usersnum=len(users)
print("usersnum",usersnum)#1976
useridmap = pd.Series(data=np.arange(usersnum), index=users)#把itemID转换成int map
userrightmap= pd.DataFrame({"UserId":users, 'UserIdx':useridmap[users].values})
#print("rightmap",rightmap)
valid = pd.merge(valid,userrightmap, on="UserId", how='left')#left??
del(valid["ItemId"])
del(valid["UserId"])
del(valid["Unnamed: 0"])
del(valid["Unnamed: 0.1"])

itemsnum 2321
usersnum 1749
itemsnum 2321
usersnum 1749


In [9]:

"""2.train_set_y生成"""
train_set_y=data.Time.values

"""3.test_set_y生成"""
test_set_y=valid.Time.values

## Trainning

In [28]:
traning=True
k = 128
model1 = Sequential()
model1.add(Embedding(usersnum + 1, k, input_length = 1))
model1.add(Reshape((k,)))
model2 = Sequential()
model2.add(Embedding(itemsnum + 1, k, input_length = 1))
model2.add(Reshape((k,)))
model3 = Sequential()
model3.add(Dense(1,input_shape=(1,)))
model4 = Sequential()
model4.add(Dense(1,input_shape=(1,)))

model = Sequential()
model.add(Merge([model1, model2,model3,model4], mode = 'concat'))
model.add(Dropout(0.2))
model.add(Dense(k, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(int(k/4), activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(int(k/16), activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation = 'linear'))
model.compile(loss = 'mse', optimizer = "adam")

X_train = [data.UserIdx.values,data.ItemIdx.values,data.Quantity.values,data.Time.values ]
X_test=[valid.UserIdx.values,valid.ItemIdx.values, valid.Quantity.values,valid.Time.values]


y_train = train_set_y
y_test= test_set_y
#allList=[]
print ("X_train: " ,X_train)
print("X_test:",X_test)
print ("y_train: " , y_train)
print ("y_test : " , y_test)


#print("X_train",X_train.shape)
#print("X_test",X_test.shape)
print("y_train",y_train.shape)
print("y_test",y_test.shape)
if traning:
    model.compile(loss='mean_squared_error', optimizer='adam',metrics=['mae'])

    model.fit(X_train, y_train,epochs=3000,batch_size=10000,validation_data=(X_test, y_test) )
    score = model.evaluate(X_test, y_test)
    print(score)
    json_string = model.to_json()  #等价于 json_string = model.get_config()  
    open('my_model_architecture.json','w').write(json_string)    
    model.save_weights('my_model_weights.h5')    
    
    #model.save_weights('EmbeddingModeldeepW.hdf5')

D:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:15: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  from ipykernel import kernelapp as app


X_train:  [array([   0,    0,    0, ..., 1747, 1747, 1748]), array([   0,    0,    1, ...,  555,  555, 1616]), array([10, 10, 12, ...,  2,  6, 60], dtype=int64), array([ 187.7625    ,   90.15069444,   49.98125   , ...,  106.04930556,
          9.31875   ,    0.        ])]
X_test: [array([   0,    0,    0, ..., 1747, 1747, 1748]), array([   0,    1,    2, ...,  554,  555, 1616]), array([10, 12, 24, ...,  2,  1, 12], dtype=int64), array([  37.14375   ,   37.14375   ,   90.15069444, ...,   12.94097222,
         20.73611111,  142.98888889])]
y_train:  [ 187.7625       90.15069444   49.98125    ...,  106.04930556    9.31875
    0.        ]
y_test :  [  37.14375      37.14375      90.15069444 ...,   12.94097222   20.73611111
  142.98888889]
y_train (66092,)
y_test (26406,)
Train on 66092 samples, validate on 26406 samples
Epoch 1/3000
66092/66092 [==============================] - 0s - loss: 5055.6823 - mean_absolute_error: 47.0864 - val_loss: 6938.7506 - val_mean_absolute_error: 58.5691
Epo

66092/66092 [==============================] - 0s - loss: 1447.6367 - mean_absolute_error: 21.6304 - val_loss: 1066.5043 - val_mean_absolute_error: 23.7104
Epoch 93/3000
66092/66092 [==============================] - 0s - loss: 1430.3774 - mean_absolute_error: 21.5788 - val_loss: 1111.3101 - val_mean_absolute_error: 24.1802
Epoch 94/3000
66092/66092 [==============================] - 0s - loss: 1440.7745 - mean_absolute_error: 21.5560 - val_loss: 984.6085 - val_mean_absolute_error: 22.6919
Epoch 95/3000
66092/66092 [==============================] - 0s - loss: 1413.3488 - mean_absolute_error: 21.4087 - val_loss: 1081.9290 - val_mean_absolute_error: 23.8231
Epoch 96/3000
66092/66092 [==============================] - 0s - loss: 1418.3254 - mean_absolute_error: 21.5751 - val_loss: 1022.9814 - val_mean_absolute_error: 23.1197
Epoch 97/3000
66092/66092 [==============================] - 0s - loss: 1436.3237 - mean_absolute_error: 21.6151 - val_loss: 1101.9057 - val_mean_absolute_error: 24.

66092/66092 [==============================] - 0s - loss: 1353.3109 - mean_absolute_error: 21.0747 - val_loss: 944.1732 - val_mean_absolute_error: 22.0769
Epoch 189/3000
66092/66092 [==============================] - 0s - loss: 1357.1288 - mean_absolute_error: 21.0022 - val_loss: 1065.8891 - val_mean_absolute_error: 23.6089
Epoch 190/3000
66092/66092 [==============================] - 0s - loss: 1343.4356 - mean_absolute_error: 20.9480 - val_loss: 942.8758 - val_mean_absolute_error: 22.0677
Epoch 191/3000
66092/66092 [==============================] - 0s - loss: 1379.2450 - mean_absolute_error: 21.1439 - val_loss: 1003.4790 - val_mean_absolute_error: 22.9139
Epoch 192/3000
66092/66092 [==============================] - 0s - loss: 1375.6709 - mean_absolute_error: 21.0108 - val_loss: 967.6952 - val_mean_absolute_error: 22.4581
Epoch 193/3000
66092/66092 [==============================] - 0s - loss: 1367.2177 - mean_absolute_error: 21.0714 - val_loss: 1054.5514 - val_mean_absolute_error: 

66092/66092 [==============================] - 0s - loss: 1339.4777 - mean_absolute_error: 20.6211 - val_loss: 1048.3919 - val_mean_absolute_error: 23.1983
Epoch 285/3000
66092/66092 [==============================] - 0s - loss: 1326.6022 - mean_absolute_error: 20.6273 - val_loss: 954.8249 - val_mean_absolute_error: 22.0260
Epoch 286/3000
66092/66092 [==============================] - 0s - loss: 1341.0010 - mean_absolute_error: 20.6841 - val_loss: 961.8350 - val_mean_absolute_error: 22.3428
Epoch 287/3000
66092/66092 [==============================] - 0s - loss: 1375.9082 - mean_absolute_error: 20.9066 - val_loss: 1015.4799 - val_mean_absolute_error: 22.9758
Epoch 288/3000
66092/66092 [==============================] - 0s - loss: 1326.2213 - mean_absolute_error: 20.6534 - val_loss: 1008.3554 - val_mean_absolute_error: 22.8473
Epoch 289/3000
66092/66092 [==============================] - 0s - loss: 1348.6536 - mean_absolute_error: 20.7898 - val_loss: 1007.5035 - val_mean_absolute_error:

...

66092/66092 [==============================] - 0s - loss: 1003.5695 - mean_absolute_error: 17.6442 - val_loss: 805.2016 - val_mean_absolute_error: 20.1152
Epoch 2584/3000
66092/66092 [==============================] - 0s - loss: 986.1983 - mean_absolute_error: 17.5382 - val_loss: 825.7338 - val_mean_absolute_error: 20.2013
Epoch 2585/3000
66092/66092 [==============================] - 0s - loss: 963.4881 - mean_absolute_error: 17.5053 - val_loss: 762.4652 - val_mean_absolute_error: 19.4766
Epoch 2586/3000
66092/66092 [==============================] - 0s - loss: 982.4278 - mean_absolute_error: 17.4534 - val_loss: 856.4600 - val_mean_absolute_error: 20.7007
Epoch 2587/3000
66092/66092 [==============================] - 0s - loss: 974.7407 - mean_absolute_error: 17.4958 - val_loss: 823.6241 - val_mean_absolute_error: 20.1215
Epoch 2588/3000
66092/66092 [==============================] - 0s - loss: 989.5916 - mean_absolute_error: 17.5657 - val_loss: 772.1382 - val_mean_absolute_error: 19.

66092/66092 [==============================] - 0s - loss: 1009.4952 - mean_absolute_error: 17.5963 - val_loss: 767.0654 - val_mean_absolute_error: 19.3114
Epoch 2680/3000
66092/66092 [==============================] - 0s - loss: 993.2298 - mean_absolute_error: 17.5376 - val_loss: 830.4378 - val_mean_absolute_error: 20.2503
Epoch 2681/3000
66092/66092 [==============================] - 0s - loss: 995.9569 - mean_absolute_error: 17.5733 - val_loss: 783.4470 - val_mean_absolute_error: 19.8168
Epoch 2682/3000
66092/66092 [==============================] - 0s - loss: 971.3401 - mean_absolute_error: 17.4524 - val_loss: 813.3475 - val_mean_absolute_error: 20.1363
Epoch 2683/3000
66092/66092 [==============================] - 0s - loss: 978.5809 - mean_absolute_error: 17.5219 - val_loss: 822.7572 - val_mean_absolute_error: 20.2328
Epoch 2684/3000
66092/66092 [==============================] - 0s - loss: 983.8844 - mean_absolute_error: 17.4279 - val_loss: 831.6286 - val_mean_absolute_error: 20.

66092/66092 [==============================] - 0s - loss: 984.2913 - mean_absolute_error: 17.4300 - val_loss: 815.9321 - val_mean_absolute_error: 19.8304
Epoch 2776/3000
66092/66092 [==============================] - 0s - loss: 991.0373 - mean_absolute_error: 17.4872 - val_loss: 856.1318 - val_mean_absolute_error: 20.4913
Epoch 2777/3000
66092/66092 [==============================] - 0s - loss: 979.5783 - mean_absolute_error: 17.4261 - val_loss: 741.3934 - val_mean_absolute_error: 19.2578
Epoch 2778/3000
66092/66092 [==============================] - 0s - loss: 964.8892 - mean_absolute_error: 17.3793 - val_loss: 846.8173 - val_mean_absolute_error: 20.3036
Epoch 2779/3000
66092/66092 [==============================] - 0s - loss: 981.2169 - mean_absolute_error: 17.4328 - val_loss: 763.0765 - val_mean_absolute_error: 19.4219
Epoch 2780/3000
66092/66092 [==============================] - 0s - loss: 984.5452 - mean_absolute_error: 17.5304 - val_loss: 804.8581 - val_mean_absolute_error: 20.0

66092/66092 [==============================] - 0s - loss: 957.8185 - mean_absolute_error: 17.2666 - val_loss: 755.3681 - val_mean_absolute_error: 19.3478
Epoch 2872/3000
66092/66092 [==============================] - 0s - loss: 968.3673 - mean_absolute_error: 17.3784 - val_loss: 814.7931 - val_mean_absolute_error: 20.0383
Epoch 2873/3000
66092/66092 [==============================] - 0s - loss: 977.0277 - mean_absolute_error: 17.4164 - val_loss: 799.8958 - val_mean_absolute_error: 19.8644
Epoch 2874/3000
66092/66092 [==============================] - 0s - loss: 955.4983 - mean_absolute_error: 17.3677 - val_loss: 797.3291 - val_mean_absolute_error: 19.8036
Epoch 2875/3000
66092/66092 [==============================] - 0s - loss: 947.7103 - mean_absolute_error: 17.2794 - val_loss: 827.0192 - val_mean_absolute_error: 20.2245
Epoch 2876/3000
66092/66092 [==============================] - 0s - loss: 972.8039 - mean_absolute_error: 17.3788 - val_loss: 803.6388 - val_mean_absolute_error: 19.8

66092/66092 [==============================] - 0s - loss: 956.8253 - mean_absolute_error: 17.3238 - val_loss: 840.3859 - val_mean_absolute_error: 20.2571
Epoch 2968/3000
66092/66092 [==============================] - 0s - loss: 965.4120 - mean_absolute_error: 17.3552 - val_loss: 823.6695 - val_mean_absolute_error: 20.0015
Epoch 2969/3000
66092/66092 [==============================] - 0s - loss: 958.2172 - mean_absolute_error: 17.2897 - val_loss: 790.3676 - val_mean_absolute_error: 19.7343
Epoch 2970/3000
66092/66092 [==============================] - 0s - loss: 968.3422 - mean_absolute_error: 17.4024 - val_loss: 802.8031 - val_mean_absolute_error: 19.8982
Epoch 2971/3000
66092/66092 [==============================] - 0s - loss: 948.5372 - mean_absolute_error: 17.2679 - val_loss: 784.8943 - val_mean_absolute_error: 19.6748
Epoch 2972/3000
66092/66092 [==============================] - 0s - loss: 986.7177 - mean_absolute_error: 17.4213 - val_loss: 879.7379 - val_mean_absolute_error: 20.7

## Load model

In [5]:
def getUserId(UserIdx):
    return userrightmap.loc[userrightmap.loc[:,"UserIdx"]==UserIdx].UserId
def getUserIdx(UserId):
    return userrightmap.loc[userrightmap.loc[:,"UserId"]==UserId].UserIdx  
def getItemId(ItemIdx):
    return rightmap.loc[rightmap.loc[:,"ItemIdx"]==ItemIdx].ItemId
def getItemIdx(ItemId):
    #这是坑str(ItemId)， 因为ItemId里面有字符串，所以是str类型，==判断的时候放进去整数不会相等
    return rightmap.loc[rightmap.loc[:,"ItemId"]==str(ItemId)].ItemIdx


def predictSingle(userId,itemId,quantity,purchaseCycle):
    #print('userId',userId,'itemId',itemId,'quantity',quantity,"purchaseCycle",purchaseCycle)
    return model.predict([np.array(getUserIdx(userId)),np.array(getItemIdx(itemId)),np.array(quantity).reshape(1,1),np.array(purchaseCycle).reshape(1,1)])


#model.load_weights('EmbeddingModeldeepW.hdf5',by_name=True)
#加载模型数据和weights  
model = model_from_json(open('my_model_architecture.json').read())    
model.load_weights('my_model_weights.h5')   

D:\Anaconda3\envs\tensorflow\lib\site-packages\keras\engine\topology.py:1252: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  return cls(**config)


In [6]:
#print(np.array(getItemIdx(22029)))
#print(np.array(getUserIdx(12377)))
#print(userrightmap.loc[:,"UserId"]==12377)
#predictSingle(12347,22729,4,315.0569444444427)

print(predictSingle(12347,22029,12,321.0))

[[ 212.72402954]]


## Prindict 

In [8]:
reparchaseCycleList=[]
timeSetList=[]
rPCList=[]
#时间函数

  

###########################################################################
def RPTimeFuctionNN(alfa,beta,aUsersItemTimes,aUsersItemQuantity,itemKey,NowTimestampTestSet,userId,TimeDelay):
    daltaTsum=0
    preTime=-1
    BaseP=beta
   
    if itemKey not in aUsersItemTimes.keys():#判断是否买过
        return BaseP   
    aItemTimes=aUsersItemTimes[itemKey]#aItemTimes:[100,200,300]
    aItemQuantity=aUsersItemQuantity[itemKey]
    prechaseTimes=len(aItemTimes)
    if(prechaseTimes<2):
        return BaseP
    else:
        try: #防止[0][0]取不到数据
            reparchaseCycle=predictSingle(userId,itemKey,aItemQuantity[-1],NowTimestampTestSet-aItemTimes[-1])[0][0]
        except:
             return BaseP
    if(reparchaseCycle!=0):
        reparchaseCycleList.append(NowTimestampTestSet-aItemTimes[-1]-reparchaseCycle)
        timeSetList.append(NowTimestampTestSet-aItemTimes[-1])
        rPCList.append(reparchaseCycle)
    if reparchaseCycle==0:
        return BaseP
    
    #NowTimestampTestSet=recommendDate=NextBuyDate
    #print("NowTimestampTestSet",NowTimestampTestSet,"aItemTimes[-1]",aItemTimes[-1])
    result=(1-BaseP)*np.exp(-(np.square(NowTimestampTestSet+TimeDelay+0*reparchaseCycle-aItemTimes[-1]-reparchaseCycle))/(alfa*alfa*reparchaseCycle*reparchaseCycle))+BaseP
    #result=(1-BaseP)*np.exp(-(np.square(TimeDelay))/(alfa*alfa*(NowTimestampTestSet-aItemTimes[-1])*(NowTimestampTestSet-aItemTimes[-1])))+BaseP
    
    #print("UserID:",userId," buys itemId:",itemKey,"quantity: ",aItemQuantity," at ",aItemTimes,"reparchaseCycle:",reparchaseCycle)
    return result
###########################################################################
"""给一个user生成推荐RPTARS """                    
def userBasedRecommendTA(alfa,beta,neighbors,userId,users,usersTA,usersQuantity,K,TopN,ShowWatched,testAUserItems,typeName,TimeDelay):
   
    #[(1.0, '小明'), (0.7819967492686538, '隔壁老王')]
    #2.
    #print("userId",userId,"neighbors",neighbors)
    movieRatings={}# neigbors item rating * sim
    DIVmovieRatings={}# fen mu
    #每个邻居
    for n in neighbors:#(0.7, '小明')
        sim=n[0]#0.7
        movieDic=users[n[1]]#每个用户的全部评价字典{'movie':1,'movie2':2...} n[1]是小明
        #每个邻居的每个item
        for m in movieDic:#  item:1 每个邻居的每个item. m is key
            movieDic[m]#分数   1
            if m in movieRatings:#not first item, normalization
                movieRatings[m]=movieDic[m]*sim+movieRatings[m]
                #DIVmovieRatings[m]=sim+DIVmovieRatings[m]
                DIVmovieRatings[m]=DIVmovieRatings[m]+1
            else:  # if first item
                movieRatings[m]=movieDic[m]*sim   #预测分数
                #DIVmovieRatings[m]=sim             #相似度
                DIVmovieRatings[m]=1

    #print("testAUserItems",testAUserItems)# {'23552': [15315.327777777777], '84625A': [15315.327777777777], '23508': [15315.327777777777]}
    # print("movieRatings",movieRatings)# {'20961': 0.2475368857441686, '21213': 0.25743836117393532, '23173': 1.2913767873548092, '22729': 0.32410186177608219, '22726': 0.32410186177608219, '22730': 0.32410186177608219, '23295': 0.25743836117393532, '23162': 0.232210182006412, '21232': 0.2475368857441686, '22666': 0.25743836117393532, '23172': 1.2913767873548092, '23174': 1.2913767873548092, '23296': 0.25743836117393532, '22952': 0.232210182006412, '22423': 1.2913767873548092, '22800': 0.23008949665421116, '22725': 0.32410186177608219, '22665': 0.25743836117393532, '22469': 0.2475368857441686, '21136': 0.232210182006412, '22727': 0.32410186177608219, '22699': 1.2913767873548092, '21977': 0.48964854318034734, '23313': 0.232210182006412, '23171': 1.2913767873548092, '22801': 0.23008949665421116, '23294': 0.25743836117393532, '23245': 0.55419135843029332, '84991': 0.25743836117393532, '23175': 1.0612872907005981, '47566': 0.232210182006412, '84879': 0.232210182006412, '21314': 0.23008949665421116, '22192': 0.32410186177608219, '22193': 0.32410186177608219, '23170': 1.2913767873548092, '22487': 0.2475368857441686, '22191': 0.32410186177608219, '84987': 0.232210182006412, '21974': 0.232210182006412, '22697': 1.2913767873548092, '22089': 0.232210182006412, 'M': 0.55419135843029332, '22950': 0.232210182006412, '22956': 0.25743836117393532, '22316': 0.25743836117393532, '22935': 0.25743836117393532, '48185': 0.232210182006412, '22961': 0.25743836117393532, '22468': 0.2475368857441686, '22955': 0.25743836117393532, '23356': 0.23008949665421116, '22728': 0.32410186177608219, '22698': 1.2913767873548092, '22754': 0.2475368857441686, '21975': 0.25743836117393532, '47567B': 0.25743836117393532}
    #movieRatings是当前用户的邻居的物品
    for m in movieRatings: #每个物品生成的预测分数
        movieRatings[m]=movieRatings[m]/DIVmovieRatings[m]#dictvalues [[15315.327777777777], [15315.327777777777], [15315.327777777777]]
        dictvalues=list(testAUserItems.values())#[[16604.67078275463]]#time now
        #print("dictvalues",dictvalues)        
        #乘以个TimeFunction
        if(typeName=='TARP'):
            timeFunction=RPTimeFuctionNN(alfa,beta,usersTA[userId],usersQuantity[userId],m,dictvalues[0][0],userId,TimeDelay)
        movieRatings[m]=movieRatings[m]*(1+timeFunction)
    mylist=sorted(movieRatings.items(),key=lambda x:x[1],reverse=True)
    #mylist=[]
    #mylist=movieRatings
    if ShowWatched==False:# 不推荐已经看过的
        for m in mylist:
            if m[0] in users[userId].keys():
                mylist.remove(m)
    
    #print("Top",mylist[:TopN]) 
    
    #print(neighbors)
    #print("userId",userId,"topN",mylist[:TopN])
    return mylist[:TopN]

"""给每个user生成推荐并计算准确率 """       
def userBasedRecommendAllTARP(alfa,beta,AllNeibors,users,usersTA,usersQuantity,K,TopN,ShowWatched,testUseritems,testUseritemsTA,TimeDelay):
    rightNum=0
    hasBuyNum=0#推荐中已经买过的数量
    xunhuanNum=0
    #print("testUseritemslen",len(testUseritems.keys()))
    #print("testUseritemslen",len(testUseritems.keys())) same 3779
    #遍历所有user 并生成推荐
    #print("userlen:",len( users.keys())) same 11337
    userList= sorted(users.keys())
    
    for userid in userList:
        #给单个用户生成推荐LIst
        topNRecommenderList=userBasedRecommendTA(alfa,beta,AllNeibors[userid],userid,users,usersTA,usersQuantity,K,TopN,ShowWatched,testUseritemsTA[userid],"TARP",TimeDelay)
        #print("topNRecommenderListlen",len(topNRecommenderList))
        #print("userid",userid,"topNRecommenderList",topNRecommenderList)
        #遍历推荐的内容
        for useritem in topNRecommenderList:#same
            #print("useritem",useritem)
            itemid=useritem[0]             
            try:
                #print("testUseritems[userid]",testUseritems[userid])
                #print("userid",userid,"testUseritems[userid]",testUseritems[userid])
                #判断是否推荐的内容在测试集中
                if itemid in testUseritems[userid].keys():
                   # print("INuserId",userid,"itemid",itemid)
                    xunhuanNum=xunhuanNum+1
                    rightNum=rightNum+1
                #else:
                   # print("NotInuserId",userid,"itemid",itemid)
                #判断是否推荐的内容在用户买过的列表中
                if itemid in users[userid].keys():
                    hasBuyNum=hasBuyNum+1
                   
            except:
                print("non")
                #same 0
                continue
    print("rightNumxunhuanNum",xunhuanNum)
    userNum=len(users)       
    precision=rightNum/(TopN*userNum)
  
    recall=rightNum/(userNum*3)
   
    print("testUseritems",len(testUseritems),"user*3",userNum*3)
    F1=2*precision*recall/(precision+recall)
    print ("TARP:")
    print ("precision=",precision,"rightNum",rightNum,"recommend",TopN*len(users))    
    print ("recall=",recall)
    print ("F1=",F1)
    
    Novelty=hasBuyNum/(TopN*userNum)
    print("hasBuyNum:",hasBuyNum,"TopN:",TopN,"userNum:",userNum)
    print("Novelt=",Novelty)
    return 
    

def getNeighborsFormFile():
    FilePath="processeddata/AllNeigborsCos.csv"
    rating=pd.read_csv(FilePath)
    AllNeighbors={}#{"user1":[(1,老王),(0.74,老李)],"user2":}
    preUserId=0
    dictList=[] #[(1,老王),(0.74,老李)]
    for index,userId,similar,neighborId in rating.values:
        
        if userId==preUserId:# same user
            dictList.append((similar,neighborId))
        else: #new user
            if preUserId==0: #first user
                dictList.append((similar,neighborId))
            else: # new user
                AllNeighbors[preUserId]=dictList #add preverious user
                
                dictList=[]
                dictList.append((similar,neighborId))
    
        preUserId=userId
    AllNeighbors[preUserId]=dictList #last user 
    #print (AllNeighbors)
    return AllNeighbors

"""读取无时间的数据"""
def loadUserRatingData(rating):
    preUserId=-1
    userMovieDic={}
    ratingdict={}#每个用户各自的打分
    for index2,InvoiceNo,itemid,Description,Quantity,timestamp,userId in rating.values:
        
        #如果第一次循环
        if preUserId==-1:
            ratingdict[itemid]=1
            userMovieDic[userId]=ratingdict
        else:
            #如果接下来还是上一个用户的
            if(userId==preUserId):
                ratingdict[itemid]=1
            #如果接下来是新用户的
            if(userId!=preUserId):
                #把上个用户的信息存到大字典里
                userMovieDic[preUserId]=ratingdict
                #清空上个用户信息
                ratingdict={}#清空
                #保存这个用户信息
                ratingdict[itemid]=1
        #赋当前用户名值给旧用户名
        preUserId=userId
    #print(userMovieDic)

    return userMovieDic
def loadUserRatingDataTime(rating,dataset):
    preUserId=-1
    
    userMovieDic={}
    ratingdict={}#每个用户各自的打分
    timeList=[]#每个用户对应每个物品的打分
    alltimeListInDay=[]
    for  index,InvoiceNo,itemid,Description,Quantity,timestamp,userId in rating.values:
        #如果第一次循环
        #timeArray = time.strptime(timestamp, "%m/%d/%Y %H:%M")
        #转换为时间戳:
        #timestamp = int(time.mktime(timeArray))
        #userMovieDic={"user1":{"item1":[10d,100d,200d],"item2":[10d,20d]},[]}    
        timestamp=timestamp/3600/24
        alltimeListInDay.append(timestamp)
        #print("timeStamp",timestamp) 
        if preUserId==-1:
            
            timeList.append(timestamp)
            ratingdict[itemid]=timeList
            userMovieDic[userId]=ratingdict
        else:
            #如果接下来还是上一个用户的
            if userId==preUserId :
                if itemid in ratingdict.keys():#if user's item list already have this item
                    timeList=ratingdict[itemid]#append the time ti time list
                else: # user's item list do not have this item yet
                    timeList=[]#clear the time list
                timeList.append(timestamp)
                ratingdict[itemid]=timeList
                
            #如果接下来是新用户的
            if userId!=preUserId :
                
                
                #把上个用户的信息存到大字典里
                userMovieDic[preUserId]=ratingdict
                #清空上个用户信息
                ratingdict={}#清空
                timeList=[]
                #保存这个用户信息
                timeList.append(timestamp)
                ratingdict[itemid]=timeList                
                
        #赋当前用户名值给旧用户名
        preUserId=userId
        
    #print(userMovieDic)
    if dataset=='test':#测试集输出时间分布
        #ppl.boxplot(alltimeListInDay)
        
        #plt.show()        
        print("test time mean:",np.mean(alltimeListInDay))
    return userMovieDic

def loadUserRatingQuantity(rating):
    preUserId=-1
    
    userMovieDic={}
    ratingdict={}#每个用户各自的打分
    quantityList=[]#每个用户对应每个物品的打分
    allquantityListInDay=[]
    for  index,InvoiceNo,itemid,Description,Quantity,timestamp,userId in rating.values:
        allquantityListInDay.append(Quantity)
        if preUserId==-1:
            quantityList.append(Quantity)
            ratingdict[itemid]=quantityList
            userMovieDic[userId]=ratingdict
        else:
            #如果接下来还是上一个用户的
            if userId==preUserId :
                if itemid in ratingdict.keys():#if user's item list already have this item
                    quantityList=ratingdict[itemid]#append the time to time list
                    #print("quantityList",quantityList)
                else: # user's item list do not have this item yet
                    quantityList=[]#clear the time list
                quantityList.append(Quantity)
                ratingdict[itemid]=quantityList
                
            #如果接下来是新用户的
            if userId!=preUserId :
                
                
                #把上个用户的信息存到大字典里
                userMovieDic[preUserId]=ratingdict
                #清空上个用户信息
                ratingdict={}#清空
                quantityList=[]
                #保存这个用户信息
                quantityList.append(Quantity)
                ratingdict[itemid]=quantityList                
                
        #赋当前用户名值给旧用户名
        preUserId=userId
        
    #print(userMovieDic)
    
    return userMovieDic



## NN-PC with Different Top_N

In [9]:
print("start load")
ratingTrain=pd.read_csv("processeddata/dataOrderMoreThan10TrainT.csv")
ratingTest=pd.read_csv("processeddata/dataOrderMoreThan10TestT.csv")
userMovieDicTraningset=loadUserRatingData(ratingTrain)
userMovieDicTestset=loadUserRatingData(ratingTest)

userMovieDicTraningTA=loadUserRatingDataTime(ratingTrain,"train")
userMovieDicTestsetTA=loadUserRatingDataTime(ratingTest,"test")

#加载进来数量
userMovieDicTraningQuantity=loadUserRatingQuantity(ratingTrain)
userMovieDicTestsetQuantity=loadUserRatingQuantity(ratingTest)

AllNeibors=getNeighborsFormFile()
print("end load")
#userBasedRecommendAllTARP(0.2,0.0,AllNeibors,userMovieDicTraningset,userMovieDicTraningTA,userMovieDicTraningQuantity,5,10,True,userMovieDicTestset,userMovieDicTestsetTA)

#TimeDelay=0
#userBasedRecommendAllTARP(0.06,0.0,AllNeibors,userMovieDicTraningset,userMovieDicTraningTA,userMovieDicTraningQuantity,5,10,True,userMovieDicTestset,userMovieDicTestsetTA,TimeDelay)
for i in range(1,4):
    TimeDelay=0
    print("Top-N",i*5)
    userBasedRecommendAllTARP(0.08,0.0,AllNeibors,userMovieDicTraningset,userMovieDicTraningTA,userMovieDicTraningQuantity,5,5*i,True,userMovieDicTestset,userMovieDicTestsetTA,TimeDelay)
#TimeDelay=0
#userBasedRecommendAllTARP(0.06,0.0,AllNeibors,userMovieDicTraningset,userMovieDicTraningTA,userMovieDicTraningQuantity,5,10,True,userMovieDicTestset,userMovieDicTestsetTA,TimeDelay)
print("MAE",np.mean(np.abs(reparchaseCycleList)))#84.5062543463
print("timesetMean",np.mean(timeSetList))#63.9234334234timeSetList
print("rPCMean",np.mean(rPCList))#79.860528088#rPCList是预测的PC
print("ME",np.mean(reparchaseCycleList))#-15.9370946646

start load
test time mean: 15235.3776682
end load
Top-N 5
rightNumxunhuanNum 185
testUseritems 3779 user*3 11337
TARP:
precision= 0.009790949986768986 rightNum 185 recommend 18895
recall= 0.016318249977948313
F1= 0.012238687483461234
hasBuyNum: 3892 TopN: 5 userNum: 3779
Novelt= 0.20598041810002646
Top-N 10
rightNumxunhuanNum 357
testUseritems 3779 user*3 11337
TARP:
precision= 0.00944694363588251 rightNum 357 recommend 37790
recall= 0.031489812119608364
F1= 0.014533759439819246
hasBuyNum: 7402 TopN: 10 userNum: 3779
Novelt= 0.19587192378936227
Top-N 15
rightNumxunhuanNum 544
testUseritems 3779 user*3 11337
TARP:
precision= 0.009596895122166358 rightNum 544 recommend 56685
recall= 0.04798447561083179
F1= 0.015994825203610598
hasBuyNum: 10902 TopN: 15 userNum: 3779
Novelt= 0.19232601217253242
MAE 25.2664411179
timesetMean 67.201895889
rPCMean 50.4038
ME 16.7981358595


## NN-PC With different hyper-parameter a

In [31]:
print("start load")
ratingTrain=pd.read_csv("processeddata/dataOrderMoreThan10TrainT.csv")
ratingTest=pd.read_csv("processeddata/dataOrderMoreThan10TestT.csv")
userMovieDicTraningset=loadUserRatingData(ratingTrain)
userMovieDicTestset=loadUserRatingData(ratingTest)

userMovieDicTraningTA=loadUserRatingDataTime(ratingTrain,"train")
userMovieDicTestsetTA=loadUserRatingDataTime(ratingTest,"test")

#加载进来数量
userMovieDicTraningQuantity=loadUserRatingQuantity(ratingTrain)
userMovieDicTestsetQuantity=loadUserRatingQuantity(ratingTest)
"""
userMovieDicTraningset=loadUserRatingDataFromMovL(ratingTrain)
userMovieDicTestset=loadUserRatingDataFromMovL(ratingTest)
userMovieDicTraningTA=loadUserRatingDataTimeFromMovL(ratingTrain)
userMovieDicTestsetTA=loadUserRatingDataTimeFromMovL(ratingTest)
"""
AllNeibors=getNeighborsFormFile()
print("end load")
#userBasedRecommendAllTARP(0.2,0.0,AllNeibors,userMovieDicTraningset,userMovieDicTraningTA,userMovieDicTraningQuantity,5,10,True,userMovieDicTestset,userMovieDicTestsetTA)
for i in range(11):
    print("a=",0.01*i)
    TimeDelay=0
    userBasedRecommendAllTARP(0.01*i,0.0,AllNeibors,userMovieDicTraningset,userMovieDicTraningTA,userMovieDicTraningQuantity,5,10,True,userMovieDicTestset,userMovieDicTestsetTA,TimeDelay)
#for i in range(11):
   
    #TimeDelay=10*i
   # print("TimeDelay",TimeDelay)
#TimeDelay=0
#userBasedRecommendAllTARP(0.06,0.0,AllNeibors,userMovieDicTraningset,userMovieDicTraningTA,userMovieDicTraningQuantity,5,10,True,userMovieDicTestset,userMovieDicTestsetTA,TimeDelay)
print("MAE",np.mean(np.abs(reparchaseCycleList)))#84.5062543463
print("timesetMean",np.mean(timeSetList))#63.9234334234timeSetList
print("rPCMean",np.mean(rPCList))#79.860528088#rPCList是预测的PC
print("ME",np.mean(reparchaseCycleList))#-15.9370946646

start load
test time mean: 15235.3776682
end load
a= 0.0


D:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:35: RuntimeWarning: divide by zero encountered in double_scalars


rightNumxunhuanNum 281
testUseritems 3779 user*3 11337
TARP:
precision= 0.007435829584546176 rightNum 281 recommend 37790
recall= 0.02478609861515392
F1= 0.011439737822378733
hasBuyNum: 5549 TopN: 10 userNum: 3779
Novelt= 0.14683778777454354
a= 0.01
rightNumxunhuanNum 283
testUseritems 3779 user*3 11337
TARP:
precision= 0.007488753638528712 rightNum 283 recommend 37790
recall= 0.024962512128429037
F1= 0.011521159443890325
hasBuyNum: 5608 TopN: 10 userNum: 3779
Novelt= 0.1483990473670283
a= 0.02
rightNumxunhuanNum 287
testUseritems 3779 user*3 11337
TARP:
precision= 0.007594601746493781 rightNum 287 recommend 37790
recall= 0.02531533915497927
F1= 0.01168400268691351
hasBuyNum: 5759 TopN: 10 userNum: 3779
Novelt= 0.15239481344270972
a= 0.03
rightNumxunhuanNum 295
testUseritems 3779 user*3 11337
TARP:
precision= 0.007806297962423922 rightNum 295 recommend 37790
recall= 0.02602099320807974
F1= 0.01200968917295988
hasBuyNum: 5905 TopN: 10 userNum: 3779
Novelt= 0.15625826938343476
a= 0.04
ri

## NN-PC with diiferent Time Delay

In [32]:
"""NN的TimeDelay结果"""
print("start load")
ratingTrain=pd.read_csv("processeddata/dataOrderMoreThan10TrainT.csv")
ratingTest=pd.read_csv("processeddata/dataOrderMoreThan10TestT.csv")
userMovieDicTraningset=loadUserRatingData(ratingTrain)
userMovieDicTestset=loadUserRatingData(ratingTest)
userMovieDicTraningTA=loadUserRatingDataTime(ratingTrain,"train")
userMovieDicTestsetTA=loadUserRatingDataTime(ratingTest,"test")

userMovieDicTraningQuantity=loadUserRatingQuantity(ratingTrain)
userMovieDicTestsetQuantity=loadUserRatingQuantity(ratingTest)
"""
userMovieDicTraningset=loadUserRatingDataFromMovL(ratingTrain)
userMovieDicTestset=loadUserRatingDataFromMovL(ratingTest)
userMovieDicTraningTA=loadUserRatingDataTimeFromMovL(ratingTrain)
userMovieDicTestsetTA=loadUserRatingDataTimeFromMovL(ratingTest)
"""
AllNeibors=getNeighborsFormFile()
print("end load")
#userBasedRecommendAllTARP(0.2,0.0,AllNeibors,userMovieDicTraningset,userMovieDicTraningTA,userMovieDicTraningQuantity,5,10,True,userMovieDicTestset,userMovieDicTestsetTA)
for i in range(11):
   
    TimeDelay=10*i
    print("TimeDelay",TimeDelay)
    userBasedRecommendAllTARP(0.06,0.0,AllNeibors,userMovieDicTraningset,userMovieDicTraningTA,userMovieDicTraningQuantity,5,10,True,userMovieDicTestset,userMovieDicTestsetTA,TimeDelay)
    print("MAE",np.mean(np.abs(reparchaseCycleList)))#84.5062543463
    print("timesetMean",np.mean(timeSetList))#63.9234334234timeSetList
    print("rPCMean",np.mean(rPCList))#79.860528088#rPCList是预测的PC
    print("ME",np.mean(reparchaseCycleList))#-15.9370946646

start load
test time mean: 15235.3776682
end load
TimeDelay 0
rightNumxunhuanNum 320
testUseritems 3779 user*3 11337
TARP:
precision= 0.00846784863720561 rightNum 320 recommend 37790
recall= 0.0282261621240187
F1= 0.013027459441854784
hasBuyNum: 6571 TopN: 10 userNum: 3779
Novelt= 0.17388197935961894
MAE 25.2724625665
timesetMean 67.2277786598
rPCMean 50.4414
ME 16.786375779
TimeDelay 10
rightNumxunhuanNum 310
testUseritems 3779 user*3 11337
TARP:
precision= 0.008203228367292934 rightNum 310 recommend 37790
recall= 0.027344094557643115
F1= 0.012620351334296821
hasBuyNum: 8444 TopN: 10 userNum: 3779
Novelt= 0.22344535591426304
MAE 25.2724625665
timesetMean 67.2277786598
rPCMean 50.4414
ME 16.786375779
TimeDelay 20
rightNumxunhuanNum 281
testUseritems 3779 user*3 11337
TARP:
precision= 0.007435829584546176 rightNum 281 recommend 37790
recall= 0.02478609861515392
F1= 0.011439737822378733
hasBuyNum: 5990 TopN: 10 userNum: 3779
Novelt= 0.15850754167769252
MAE 25.2724625665
timesetMean 67.22

In [14]:
"""精确的的TimeDelay结果"""
print("start load")
ratingTrain=pd.read_csv("processeddata/dataOrderMoreThan10TrainT.csv")
ratingTest=pd.read_csv("processeddata/dataOrderMoreThan10TestT.csv")
userMovieDicTraningset=loadUserRatingData(ratingTrain)
userMovieDicTestset=loadUserRatingData(ratingTest)
userMovieDicTraningTA=loadUserRatingDataTime(ratingTrain,"train")
userMovieDicTestsetTA=loadUserRatingDataTime(ratingTest,"test")

userMovieDicTraningQuantity=loadUserRatingQuantity(ratingTrain)
userMovieDicTestsetQuantity=loadUserRatingQuantity(ratingTest)
"""
userMovieDicTraningset=loadUserRatingDataFromMovL(ratingTrain)
userMovieDicTestset=loadUserRatingDataFromMovL(ratingTest)
userMovieDicTraningTA=loadUserRatingDataTimeFromMovL(ratingTrain)
userMovieDicTestsetTA=loadUserRatingDataTimeFromMovL(ratingTest)
"""
AllNeibors=getNeighborsFormFile()
print("end load")
#userBasedRecommendAllTARP(0.2,0.0,AllNeibors,userMovieDicTraningset,userMovieDicTraningTA,userMovieDicTraningQuantity,5,10,True,userMovieDicTestset,userMovieDicTestsetTA)
for i in range(11):
   
    TimeDelay=10*i
    print("TimeDelay",TimeDelay)
    userBasedRecommendAllTARP(0.06,0.0,AllNeibors,userMovieDicTraningset,userMovieDicTraningTA,userMovieDicTraningQuantity,5,10,True,userMovieDicTestset,userMovieDicTestsetTA,TimeDelay)
    print("MAE",np.mean(np.abs(reparchaseCycleList)))#84.5062543463
    print("timesetMean",np.mean(timeSetList))#63.9234334234timeSetList
    print("rPCMean",np.mean(rPCList))#79.860528088#rPCList是预测的PC
    print("ME",np.mean(reparchaseCycleList))#-15.9370946646

start load
test time mean: 15235.3776682
end load
TimeDelay 0


D:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in true_divide


rightNumxunhuanNum 762
testUseritems 3779 user*3 11337
TARP:
precision= 0.02016406456734586 rightNum 762 recommend 37790
recall= 0.06721354855781952
F1= 0.03102163779591671
hasBuyNum: 14070 TopN: 10 userNum: 3779
Novelt= 0.37232071976713416
MAE 28.960135622
timesetMean 67.2284921278
rPCMean 54.3418
ME 12.886663525
TimeDelay 10


D:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:35: RuntimeWarning: divide by zero encountered in true_divide


rightNumxunhuanNum 451
testUseritems 3779 user*3 11337
TARP:
precision= 0.011934374173061656 rightNum 451 recommend 37790
recall= 0.039781247243538854
F1= 0.01836057565086409
hasBuyNum: 11614 TopN: 10 userNum: 3779
Novelt= 0.3073299814765811
MAE 28.960135622
timesetMean 67.2284921278
rPCMean 54.3418
ME 12.886663525
TimeDelay 20
rightNumxunhuanNum 326
testUseritems 3779 user*3 11337
TARP:
precision= 0.008626620799153216 rightNum 326 recommend 37790
recall= 0.028755402663844052
F1= 0.013271724306389565
hasBuyNum: 8791 TopN: 10 userNum: 3779
Novelt= 0.23262767928023287
MAE 28.960135622
timesetMean 67.2284921278
rPCMean 54.3418
ME 12.886663525
TimeDelay 30
rightNumxunhuanNum 292
testUseritems 3779 user*3 11337
TARP:
precision= 0.007726911881450119 rightNum 292 recommend 37790
recall= 0.025756372938167064
F1= 0.011887556740692492
hasBuyNum: 6907 TopN: 10 userNum: 3779
Novelt= 0.18277322042868482
MAE 28.960135622
timesetMean 67.2284921278
rPCMean 54.3418
ME 12.886663525
TimeDelay 40
rightNum

In [21]:

import prettyplotlib as ppl
import numpy as np


# prettyplotlib imports 
import matplotlib.pyplot as plt
import matplotlib as mpl
from prettyplotlib import brewer2mpl
import pandas as pd

import datetime 

def drawPredict(): 
    x = np.arange(len(rPCList))
    #x = np.arange(20)
    y = timeSetList
    ppl.plot( x, y, linewidth=0.75)
    
    newrPCList=np.array(rPCList).reshape(52749  ,)
    y = newrPCList
    ppl.plot( x, y,'--', linewidth=0.75)
    plt.show()

In [22]:
drawPredict()

D:\Anaconda3\envs\tensorflow\lib\site-packages\matplotlib\__init__.py:800: MatplotlibDeprecationWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  mplDeprecation)


ValueError: cannot reshape array of size 580239 into shape (52749,)